In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split



In [2]:
data = pd.read_csv("data_lidah.csv")

# Memisahkan output dan input
output = data.iloc[:, :2].values  # Mengambil kolom index 0 dan 1 sebagai output
input_data = data.iloc[:, 2:].values  # Mengambil kolom index 3 sampai 9 sebagai input

# Mengubah menjadi tensor PyTorch
input_tensor = torch.tensor(input_data, dtype=torch.float32)
output_tensor = torch.tensor(output, dtype=torch.float32)

print(len(input_tensor))
print(input_tensor)
print(len(output_tensor))
print(output_tensor)

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(input_tensor, output_tensor, test_size=0.01, random_state=42)



1099
tensor([[2243.3999, 2199.4099, 2409.5801,  ..., 2438.9099, 2380.2500,
         2287.3899],
        [2233.6299, 2047.9000, 2404.6899,  ..., 2429.1299, 2375.3701,
         2385.1399],
        [2233.6299, 2047.9000, 2404.6899,  ..., 2429.1299, 2375.3701,
         2385.1399],
        ...,
        [4975.5601, 3558.1599,  904.2000,  ...,  904.2000,  860.2200,
          586.5100],
        [4975.5601, 3567.9399,  860.2200,  ...,  860.2200,  816.2300,
          537.6300],
        [4975.5601, 3587.4900,  889.5400,  ...,  889.5400,  840.6600,
          566.9600]])
1099
tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(7,  128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 2)
        self.activation = nn.Sigmoid()  # Output ke rentang 0-1

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.activation(self.fc5(x))  # Sigmoid untuk output (karena 2 kelas)
        return x

# Buat model
model = NeuralNetwork()

criterion = nn.BCELoss()  # Binary Cross Entropy Loss (karena output pakai Sigmoid)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

epochs = 10000  # Jumlah iterasi training
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)  # Hitung loss

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:  # Print loss setiap 10 epoch
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

with torch.no_grad():  # Tidak perlu hitung gradien saat evaluasi
    y_pred = model(X_test)
    y_pred_class = (y_pred > 0.5).float()  # Konversi ke 0 atau 1
    accuracy = (y_pred_class == y_test).sum().item() / y_test.numel() * 100
    print(f'Accuracy: {accuracy:.2f}%')

Epoch [10/10000], Loss: 25.5198
Epoch [20/10000], Loss: 25.3571
Epoch [30/10000], Loss: 25.2617
Epoch [40/10000], Loss: 25.2398
Epoch [50/10000], Loss: 25.2298
Epoch [60/10000], Loss: 25.2274
Epoch [70/10000], Loss: 25.2250
Epoch [80/10000], Loss: 25.2229
Epoch [90/10000], Loss: 25.2212
Epoch [100/10000], Loss: 25.2199
Epoch [110/10000], Loss: 25.2187
Epoch [120/10000], Loss: 25.2176
Epoch [130/10000], Loss: 25.2166
Epoch [140/10000], Loss: 25.2157
Epoch [150/10000], Loss: 25.2149
Epoch [160/10000], Loss: 25.2141
Epoch [170/10000], Loss: 25.2135
Epoch [180/10000], Loss: 25.2128
Epoch [190/10000], Loss: 25.2123
Epoch [200/10000], Loss: 25.2118
Epoch [210/10000], Loss: 25.2113
Epoch [220/10000], Loss: 25.2109
Epoch [230/10000], Loss: 25.2105
Epoch [240/10000], Loss: 25.2101
Epoch [250/10000], Loss: 25.2098
Epoch [260/10000], Loss: 25.2095
Epoch [270/10000], Loss: 25.2093
Epoch [280/10000], Loss: 25.2090
Epoch [290/10000], Loss: 25.2091
Epoch [300/10000], Loss: 25.2096
Epoch [310/10000], 

In [4]:
test_s = torch.tensor([2502.44,2311.83,2482.89,2473.12,2443.79,2380.25,2262.95], dtype=torch.float32)
prediction = model(test_s)

rounded_tensor = torch.round(prediction * 100) / 100  # Membulatkan ke 2 desimal

print("Prediksi Output:", rounded_tensor)


Prediksi Output: tensor([0.5000, 0.4900], grad_fn=<DivBackward0>)
